In [1]:
# Import required libraries and dependencies
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the data into a Pandas DataFrame and make the index the "coin_id" column.
market_data_df = pd.read_csv("Resources/crypto_market_data.csv", index_col="coin_id")

# Display sample data
market_data_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [4]:
# Generate summary statistics
market_data_df.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler=StandardScaler()

In [10]:
# Create a DataFrame with the scaled data
df = pd.DataFrame(scaler.fit_transform(market_data_df), columns=market_data_df.columns, index=market_data_df.index)

# Copy the crypto names from the original data
market_data_df = market_data_df.reset_index()
df['coin_id'] = market_data_df['coin_id']


# Set the coinid column as index
coinid = df.index

# Display sample data
print(df.head(5))

              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

### Find the Best Value for k Using the Original Scaled DataFrame.

In [11]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k_values = range(1, 11)

# Create an empty list to store the inertia values
empty_list = [inertia]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using the scaled DataFrame
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(df)
    empty_list.append(model.inertia_)


# Create a dictionary with the data to plot the Elbow curve
dict = {"k": k_values, "inertia": empty_list}  


# Create a DataFrame with the data to plot the Elbow curve
df = pd.DataFrame(dict)

# Display the DataFrame
print(df)

NameError: name 'inertia' is not defined

In [12]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
plot = df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k_values, xlabel="Number of clusters", ylabel="Inertia")

AttributeError: 'DataFrame' object has no attribute 'hvplot'

#### Answer the following question: 
**Question:** What is the best value for `k`?

**Answer:**

### Cluster Cryptocurrencies with K-means Using the Original Scaled Data.

In [13]:
# Initialize the K-Means model using the best value for k
best_k = 3
kmeans = KMeans(n_clusters=best_k, random_state=42)

In [16]:
# Fit the K-Means model using the scaled data
kmeans.fit(X_scaled)

NameError: name 'X_scaled' is not defined

: 

In [1]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predictions = kmeans.predict(X_scaled)

# View the resulting array of cluster values.
arr = pd.DataFrame(predictions, columns=["class"])

NameError: name 'kmeans' is not defined

In [2]:
# Create a copy of the DataFrame
df = df.copy()

NameError: name 'df' is not defined

In [3]:
# Add a new column to the DataFrame with the predicted clusters
df["class"] = arr

# Display sample data
display(df.head(10))

NameError: name 'arr' is not defined

In [4]:
# Create a scatter plot using Pandas plot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Use "rainbow" for the color to better visualize the data.
x = "price_change_percentage_24h"
y = "price_change_percentage_7d"
colormap = "rainbow"

### Optimize Clusters with Principal Component Analysis.

In [5]:
# Create a PCA model instance and set `n_components=3`.
'n_components=3'

'n_components=3'

In [13]:
# Use the PCA model with `fit_transform` on the original scaled DataFrame to reduce to three principal components.
pca_crypto.explained_variance_ratio_
principal_components = pca.fit_transform(X_scaled)

# View the first five rows of the DataFrame. 
print(df.head(5))

NameError: name 'pca_crypto' is not defined

In [17]:
# Retrieve the explained variance to determine how much information  can be attributed to each principal component.
pca_crypto.explained_variance_ratio_
pca_crypto.explained_variance_ratio_ =pd.DataFrame(pca_crypto.explained_variance_ratio_, columns=["Explained Variance"])
pca_crypto.explained_variance_ratio_.hvplot.bar(title="Explained Variance", xlabel="Principal Component", ylabel="Variance", xticks=range(3))

NameError: name 'pca_crypto' is not defined

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [20]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you
varience = pca_crypto.explained_variance_ratio_
# Creating a DataFrame with the PCA data
df_pca = pd.DataFrame(
    data=principal_components, columns=["principal component 1", "principal component 2", "principal component 3"], index=coinid
)

# Copy the crypto names from the original data
coin_id,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y


# Set the coinid column as index
coinid = df.index

# Display sample data
data = df.head(5)

NameError: name 'pca_crypto' is not defined

### Find the Best Value for k Using the PCA Data

In [23]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
pca_crypto = pca(n_components=3)
principal_components = pca_crypto.fit_transform(X_scaled)

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using PCA DataFrame.
# 3. Append the model.inertia_ to the inertia list


# Create a dictionary with the data to plot the Elbow curve
x = "k"
y = "inertia"
dict = {"k": k_values, "inertia": empty_list}  


# Create a DataFrame with the data to plot the Elbow curve
df = pd.DataFrame(dict)

# Display the DataFrame
print

NameError: name 'pca' is not defined

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.


#### Answer the following questions: 
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [24]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=3, random_state=42)

NameError: name 'KMeans' is not defined

In [25]:
# Fit the K-Means model using the PCA data
kmeans.fit(principal_components)

NameError: name 'kmeans' is not defined

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
predictions = kmeans.predict(principal_components)

# Print the resulting array of cluster values.
print(predictions)

In [26]:
# Create a copy of the DataFrame with the PCA data
df = df.copy()

# Add a new column to the DataFrame with the predicted clusters
df["class"] = predictions

# Display sample data
print(df.head(10))

NameError: name 'df' is not defined

In [28]:
# Create a scatter plot using hvPlot by setting `x="PCA1"` and `y="PCA2"`. 
x = "PCA1"
y = "PCA2"
hover_cols = ["coin_id"]
by = "class"
title = "PCA1 vs PCA2"

### Determine the Weights of Each Feature on each Principal Component

In [30]:
# Use the columns from the original scaled DataFrame as the index.
pca_crypto = pd.DataFrame(principal_components, columns=["PCA1", "PCA2", "PCA3"], index=coinid)

NameError: name 'pd' is not defined

#### Answer the following question: 

* **Question:** Which features have the strongest positive or negative influence on each component? 
 
* **Answer:** 
    